In [1]:
import numpy as np
import pandas as pd
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin1')

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.init

In [2]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']

data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data['text'] = data['v2']
data['isSpam'] = data['v1']

del data['v1'], data['v2']

print(f'Data Shape: {data.shape}')
# imbalanced data
print(data['isSpam'].value_counts())
data.head()

Data Shape: (5572, 2)
0    4825
1     747
Name: isSpam, dtype: int64


,text,isSpam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [3]:
from sklearn.model_selection import train_test_split

X, y = data['text'], data['isSpam']

In [4]:
import re

def preprocess(string: str, *args, **kwargs) -> str:
    for i in range(len(X)):
        w=X[i]
        kkk = re.sub(r'[^ A-Za-z]', ' ', str(w))
        kkk = re.sub('[0-9].',' ', kkk)
        X[i]=kkk
        i+=1
    return X

In [5]:
preprocess(X)

<ipython-input-4-4d807d0f6922>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i]=kkk


0       Go until jurong point  crazy   Available only ...
1                           Ok lar    Joking wif u oni   
2       Free entry in   a wkly comp to win FA Cup fina...
3       U dun say so early hor    U c already then say   
4       Nah I don t think he goes to usf  he lives aro...
                              ...                        
5567    This is the  nd time we have tried   contact u...
5568                Will    b going to esplanade fr home 
5569    Pity    was in mood for that  So   any other s...
5570    The guy did some bitching but I acted like i d...
5571                           Rofl  Its true to its name
Name: text, Length: 5572, dtype: object

In [6]:
stop_words_list=['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 "shan't",
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 "shouldn't",
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 "wasn't",
 'we',
 'were',
 'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves']

In [7]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

def tokenize(string: str, *args, **kwargs) -> list:
    preprocess(X)
    stop_words = set(stop_words_list)
    for i in range(len(X)):
        result=[]
        word_tokens=word_tokenize(X[i])
        for w in word_tokens:
            if w not in stop_words:
                result.append(w)
        X[i]=result
        i+=1
    return X

In [8]:
tokenize(X)

<ipython-input-4-4d807d0f6922>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i]=kkk
<ipython-input-7-6a631fb7c822>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i]=result


0       [Go, jurong, point, crazy, Available, bugis, n...
1                          [Ok, lar, Joking, wif, u, oni]
2       [Free, entry, wkly, comp, win, FA, Cup, final,...
3           [U, dun, say, early, hor, U, c, already, say]
4       [Nah, I, think, goes, usf, lives, around, though]
                              ...                        
5567    [This, nd, time, tried, contact, u, U, Pound, ...
5568                [Will, b, going, esplanade, fr, home]
5569                        [Pity, mood, So, suggestions]
5570    [The, guy, bitching, I, acted, like, intereste...
5571                              [Rofl, Its, true, name]
Name: text, Length: 5572, dtype: object

In [9]:
from collections import Counter

def build_vocab(n, *args, **kwargs):
    tokenize(X)
    train_vocab=[]
    for i in range(len(X)):
        train_vocab+=X[i]
        i+=1
        
    vocab_count=Counter()
    result=[]
    for word in train_vocab:
        word=word.lower()
        if len(word)>2:
            result.append(word)
            vocab_count[word]=vocab_count[word]+1
            
    vocab_count=sorted(vocab_count.items(),key=(lambda x:x[1]), reverse=True)
    
    vocab={}
    i=2
    while i < (n):
        vocab['padding_idx']=0
        vocab['unk_idx']=1
        vocab[vocab_count[i-2][0]]=i
        i+=1
    return vocab

In [10]:
build_vocab(1000)

<ipython-input-4-4d807d0f6922>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i]=kkk
<ipython-input-7-6a631fb7c822>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i]=result


{'padding_idx': 0,
 'unk_idx': 1,
 'call': 2,
 'get': 3,
 'you': 4,
 'free': 5,
 'know': 6,
 'got': 7,
 'like': 8,
 'good': 9,
 'day': 10,
 'come': 11,
 'time': 12,
 'love': 13,
 'send': 14,
 'text': 15,
 'want': 16,
 'txt': 17,
 'one': 18,
 'going': 19,
 'need': 20,
 'home': 21,
 'how': 22,
 'stop': 23,
 'lor': 24,
 'today': 25,
 'sorry': 26,
 'see': 27,
 'still': 28,
 'back': 29,
 'reply': 30,
 'mobile': 31,
 'dont': 32,
 'but': 33,
 'take': 34,
 'tell': 35,
 'new': 36,
 'later': 37,
 'pls': 38,
 'please': 39,
 'think': 40,
 'what': 41,
 'just': 42,
 'phone': 43,
 'week': 44,
 'the': 45,
 'dear': 46,
 'well': 47,
 'your': 48,
 'much': 49,
 'great': 50,
 'night': 51,
 'claim': 52,
 'msg': 53,
 'hope': 54,
 'hey': 55,
 'and': 56,
 'wat': 57,
 'happy': 58,
 'yes': 59,
 'way': 60,
 'make': 61,
 'give': 62,
 'www': 63,
 'work': 64,
 'now': 65,
 'this': 66,
 'number': 67,
 'have': 68,
 'message': 69,
 'can': 70,
 'wan': 71,
 'prize': 72,
 'tomorrow': 73,
 'say': 74,
 'right': 75,
 'are': 7

In [16]:
def toTensor(*args, **kwargs) -> torch.LongTensor:
    encoded=[]
    for line in X:
        temp=[]
        vocab=build_vocab(1000)
        for w in line:
            try:
                temp.append(vocab[w])
            except KeyError:
                temp.append(vocab['unk_idx'])
        encoded.append(temp)
    
    max_len=max(len(l) for l in encoded)
    for line in encoded:
        if len(line)<max_len:
            line+=[vocab['padding_idx']]*(max_len-len(line))
            
    for i in range(len(X)):
        X[i]=torch.LongTensor(encoded[i])
        i+=1
    return X

In [ ]:
toTensor()

<ipython-input-4-4d807d0f6922>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i]=kkk
<ipython-input-7-6a631fb7c822>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i]=result


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,
                                                   stratify=y, test_size=0.1)

print(len(X_train), len(X_test))

In [ ]:
class SpamClassifier(torch.nn.Module):
    def __init__(self,init_bal=0):
        self.balance=init_bal
        super(SpamClassifier, self).__init__()

        self.fc=torch.nn.Linear(85,85)
        self.relu1 = nn.ReLU()
        self.rnn=torch.nn.LSTM(85,50,bias=True)
        self.dout = nn.Dropout(0.2)
        self.nn=torch.nn.Linear(50,25,bias=True)
        self.rnn2=torch.nn.LSTM(25,1)
        self.out_act = nn.Sigmoid()
        
    def forward():
        x=self.fc(X)
        x=self.relu1(x)
        x=self.rnn(X)
        x=self.dout(x)
        x=self.nn(X)
        x=self.rnn2(X)
        x=sefl.out_act(x)
        return x

In [ ]:
model = SpamClassifier()
criterion = torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)

In [ ]:
for epoch in range(100):
    avg_cost=0
    
    for X,y in zip(X_train, torch.LongTensor(y_train.values)):
        optimizer.zero_grad()
        outputs = SpamClassifier(X)
        loss = criterion(outputs,y)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished')

파이토치 모델링에서 계쏙 오류가 나서 우선 제출하고 다시 수정해보도록 하겠습니다...